背景：

    神经网络的学习，通过数值微分计算了神经网络的权重参数的梯度（严格来说，是损失函数关于权重参数的梯度）。

    数值微分虽然简单，也容易实现，但是计算速度比较慢。接下来学习一个高效计算权重参数的方法----误差反向传播法。

    理解误差反向传播法的两种方法：
     
     1、数学式
     2、 computational graph 计算图

计算图流程
     
     1、构建计算图
     2、在计算图上做正向传播：从计算图出发点到结束点的传播
     3、在计算图上做反向传播：从计算图结束点到出发点的传播

局部计算

    计算图的特征是可以通过传递“局部计算”获得最终结果。

        局部：与自己相关的某个小范围
        局部计算：无论全局发生了什么，都能只根据与自己相关的信息输出接下来的结果。

计算图优点：

计算图可以集中精力于局部计算，无论全局的计算有多么复杂，各个步骤所要做的就是对象节点的局部计算。虽然局部计算非常简单，但是通过传递它的计算结果，可以获得全局的复杂计算的结果。

利用计算图可以将中间的计算结果全部保存起来

以通过正向传播和反向传播高效地计算各个变量的导数值

反向传播中传递局部导数的原理，是基于链式法则的：

    链式法则：
    假设函数f(x)由函数g(x)和h(x)复合而成，即f(x)=g(h(x))。
    则函数f(x)的导数可以表示为：
$$
    \frac{df(x)}{dx} = \frac{df(x)}{dg(x)}\frac{dg(x)}{dh(x)}
$$
    其中，
$\frac{df(x)}{dg(x)}$表示函数f(x)对函数g(x)的导数，$\frac{dg(x)}{dh(x)}$表示函数g(x)对函数h(x)的导数。
    也就是说，要计算函数f(x)的导数，需要先计算函数g(x)的导数，再计算函数h(x)的导数，最后将两者相乘即可。

    反向传播中传递局部导数的原理：
    反向传播中，我们需要计算损失函数L关于神经网络参数的导数，即需要计算
$\frac{dL}{dW}$和$\frac{dL}{db}$等参数的导数。   




链式法则是关于复合函数的导数的性质：
      
      如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

1、太郎在超市买了2个100日元一个的苹果，消费税是10%，请计算购买苹果的总价（含税）。
2、太郎在超市买了2个苹果、3个橘子。其中，苹果每个100日元，橘子每个150日元。消费税是10%，请计算支付金额。

In [5]:
#实现乘法层，作为MulLayer类
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    #正向传播
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    #反向传播将从上游传来的导数（dout）乘以正向传播的翻转值（y和x交换位置），并传递给下游
    def backward(self, dout):
        dx = dout * self.y   #x和y交换位置
        dy = dout * self.x
        return dx, dy

In [6]:
#对于问题1,只涉及乘法
apple = 100 
apple_num = 2
tax = 1.1     #消费税
#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

#backward
dprice = 1  #损失函数对最终输出的导数
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax)
"""参数中需要输入关于正向传播时的输出变量的导数
比如,mul_apple_layer乘法层在正向传播时会输出apple_price,在反向传播时,则会将apple_price的导数dapple_price设为参数。
"""

220.00000000000003
2.2 110.00000000000001 200


'参数中需要输入关于正向传播时的输出变量的导数\n比如,mul_apple_layer乘法层在正向传播时会输出apple_price,在反向传播时,则会将apple_price的导数dapple_price设为参数。\n'

加法层不需要特意进行初始化，所以__init__()中什么也不运行（pass 语句表示“什么也不运行”）。
加法层的forward()接收x和y两个参数，将它们相加后输出。backward()将上游传来的导数（dout）原封不动地传递给下游。

In [7]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):   #backward()将上游传来的导数（dout）原封不动地传递给下游
        dx = dout * 1
        dy = dout * 1
        return dx, dy


In [8]:
# 对于问题2 ,涉及到乘法和加法
apple = 100 
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1
#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()
#forward
apple_price = mul_apple_layer.forward(apple, apple_num)  #(1)
orange_price = mul_orange_layer.forward(orange, orange_num)  #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  #(3)
price = mul_tax_layer.forward(all_price, tax)  #(4)
#backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  #(1)
print(price)
print(dapple, dapple_num, dtax, dorange, dorange_num, dprice)


715.0000000000001
2.2 110.00000000000001 650 3.3000000000000003 165.0 1


总结以上命令，首先生成必要的层，以适合的顺序调用正向传播forward()方法，然后用正向传播相反的顺序调用反向传播backward()方法，就可以求出想要的导数。
接下来来实现神经网络中使用的层。

1、激活函数的实现

1.1 ReLU函数

ReLU函数的表达式如下：

$$
h(x) = \begin{cases}
x & (x > 0) \\
0 & (x \leq 0)
\end{cases}
$$

ReLU函数的导数表达式如下：
$$
h'(x) = \begin{cases}
1 & (x > 0) \\
0 & (x \leq 0)
\end{cases}
$$
可以看出，ReLU函数的导数在自变量大于0的区域，导数恒为1；在自变量小于等于0的区域，导数恒为0。
如果正向传播时，输入x为正值，那么反向传播时，上游的值会直接传给下游。
如果正向传播时，输入x为负值，那么反向传播时，上游的值为0，反向传播传给下游的信号将停在此处。

In [13]:
#实现ReLU层，一般假定forward() 和 backward() 的参数是Numpy 数组
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None  #实例变量mask

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()         # 为了避免因直接修改x而导致的错误，这里使用copy()方法创建x的副本
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0    # 反向传播时，将dout中对应mask为True的元素设为0，其他元素保持不变
        dx = dout

        return dx
#这个变量mask是由True/False构成的NumPy数组，它会把正向传播时的输入x的元素中小于等于0的地方保存为True，其他地方（大于0的元素）保存为False。如下例所示，mask变量保存了由True/False构成的NumPy数组。
x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print (x)
#mask变量保存了由True/False构成的NumPy数组，正向传播时的输入x的元素中小于等于0的地方保存为True，其他地方（大于0的元素）保存为False。
mask=(x<=0)
print(mask)


[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


如果正向传播时的输入值小于等于0，则反向传播的值为0。因此，反向传播中会使用正向传播时保存的mask，将从上游传来的dout的mask中的元素为True的地方设为0。

1.2 Sigmoid函数

Sigmoid函数的表达式如下：
$$
h(x) = \frac{1}{1 + \exp(-x)}
$$
Sigmoid函数的导数表达式如下：
$$
h'(x) = \frac{\exp(-x)}{(1 + \exp(-x))^2} = \frac{1}{1 + \exp(-x)} \frac{\exp(-x)}{1 + \exp(-x)} = h(x)(1 - h(x))
$$

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
#这个实现中，正向传播时将输出保存在了实例变量out 中，反向传播时则使用了该变量out 进行计算。
    

Affine 层和softmax 层的实现

    1、Affine 层的实现

    神经网络的正向传播中，为了计算加权信号的总和，使用了矩阵的乘积运算（numpy 中是np.doy()）    Y=np.dox(X,W) + B  Y经过激活函数转换后，传递给下一层


In [ ]:
import numpy as np
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)      
        return dx 

Softmax-with-Loss 层

    1、softmax 层
    
        输入：神经网络的输出（10维向量）
        输出：概率分布（10维向量）
        作用：将神经网络的输出转换为概率分布，用于分类任务
        公式：
$$
    y_k = \frac{exp(a_k)}{\sum_{i=1}^{n} exp(a_i)}
$$
其中，$y_k$ 表示第 $k$ 个输出，$a_k$ 表示第 $k$ 个输出的加权信号，$n$ 表示输出层的神经元数

softmax层将输入值正规化之后再输出，分类有几个，输出就有几个，输出为分类的概率。

神经网络中进行的处理有推理（inference）和学习两个阶段。神经网络的推理通常不使用Softmax层。比如，用图5-28的网络进行推理时，会将最后一个Affine层的输出作为识别结果。神经网络中未被正规化的输出结果
有时被称为“得分”。也就是说，当神经网络的推理只需要给出一个答案的情况下，因为此时只对得分最大值感兴趣，所以不需要Softmax层。不过，神经网络的学习阶段则需要Softmax层。


    2、 交叉熵误差

     交叉熵Softmax层的反向传播得到了（y1−t1,y2−t2,y3−t3）这样“漂亮”的结果。由于（y1,y2,y3）是Softmax层的输出，（t1,t2,t3）是监督数据，所以（y1−t1,y2−t2,y3−t3）是Softmax层的输出和教师标签的差分。神经网络的反向传播会把这个差分表示的误差传递给前面的层，这是神经网络学习中的重要性质。

     比如思考教师标签是(0,1,0)，Softmax层的输出是(0.01, 0.99,0)的情形（这个神经网络识别得相当准确）。此时Softmax层的反向传播传递的是(0.01,−0.01,0)这样一个小的误差。这个小的误差也会向前面的层传播，因为误差很小，所以Softmax层前面的层学到的内容也很“小”。

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  #损失
        self.y = None     #softmax的输出
        self.t = None     #监督数据
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_error(self.y, self.t)
        return loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        return dx

这个实现利用了softmax()和cross_entropy_error()函数。因此，这里的实现非常简单。请注意反向传播时，将要传播的值除以批的大小（batch_size）后，传递给前面的层的是单个数据的误差。

误差反向传播法的实现

神经网络学习的全部过程

1、mini-batch                   从训练数据中随机选出一部分数据
2、计算梯度                      计算损失函数关于各个权重参数的梯度
3、权重参数的更新                 权重参数沿梯度方向进行微小更新
4、重复mini-batch               重复 1，2，3

和需要花费较多时间的数值微分不同，误差反向传播法可以快速高效地计算梯度。


具体代码生成见digit_recognition_21
神经网络的层保存为OrderedDict这一点非常重要。OrderedDict是有序字典，“有序”是指它可以记住向字典里添加元素的顺序。因此，神经网络的正向传播只需按照添加元素的顺序调用各层的forward()方法就可以完成处理，而反向传播只需要按照相反的顺序调用各层即可。因为Afne层和ReLU层的内部会正确处理正向传播和反向传播，所以这里要做的事情仅仅是以正确的顺序连接各层，再按顺序（或者逆序）调用各层。


到目前为止，我们介绍了两种求梯度的方法。一种是基于数值微分的方法，另一种是解析性地求解数学式的方法。后一种方法通过使用误差反向传播法，即使存在大量的参数，也可以高效地计算梯度。因此，后文将不再使用耗费时间的数值微分，而是使用误差反向传播法求梯度。

数值微分的计算很耗费时间，而且如果有误差反向传播法的（正确的）实现的话，就没有必要使用数值微分的实现了。那么数值微分有什么用呢？实际上，在确认误差反向传播法的实现是否正确时，是需要用到数值微分的。

数值微分的优点是实现简单，因此，一般情况下不太容易出错。而误差反向传播法的实现很复杂，容易出错。所以，经常会比较数值微分的结果和误差反向传播法的结果，以确认误差反向传播法的实现是否正确。确认数值微分求出的梯度结果和误差反向传播法求出的结果是否一致（严格地讲，是非常相近）的操作称为梯度确认（gradient check）。

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 求各个权重的绝对误差的平均值
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

和以前一样，读入MNIST数据集。然后，使用训练数据的一部分，确认数值微分求出的梯度和误差反向传播法求出的梯度的误差。这里误差的计算方法是求各个权重参数中对应元素的差的绝对值，并计算其平均值。运行上面的代码后，会输出如下结果。
b1:9.70418809871e-13 
W2:8.41139039497e-13 
b2:1.1945999745e-10
W1:2.2232446644e-13
从这个结果可以看出，通过数值微分和误差反向传播法求出的梯度的差非常小。比如，第1层的偏置的误差是9.7e-13（0.00000000000097）。这样一来，我们就知道了通过误差反向传播法求出的梯度是正确的，误差反向传播法的实现没有错误。

数值微分和误差反向传播法的计算结果之间的误差为0是很少见的。这是因为计算机的计算精度有限（比如，32位浮点数）。受到数值精度的限制，刚才的误差一般不会为0，但是如果实现正确的话，可以期待这个误差是一个接近0的很小的值。如果这个值很大，就说明误差反向传播法的实现存在错误。